In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [3]:
data = spark.read.csv('gs://new_bucket_vaishu/customer_churn.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [9]:
output = assembler.transform(data)
final_data = output.select('features','churn')

In [10]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [11]:
from pyspark.ml.classification import LogisticRegression

In [12]:
lr_churn = LogisticRegression(labelCol='churn')
fitted_churn_model = lr_churn.fit(train_churn)
training_sum = fitted_churn_model.summary
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               607|                607|
|   mean|0.1696869851729819|0.13179571663920922|
| stddev|0.3756671524778242|0.33854752524026677|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [14]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.68031894552534...|[0.99080919951052...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.68283741125866...|[0.66437168369246...|       0.0|
|[28.0,11128.95,1....|    0|[4.09089447421843...|[0.98355083283077...|       0.0|
|[28.0,11245.38,0....|    0|[3.77225662360167...|[0.97751700898150...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.73589973966196...|[0.99130177256760...|       0.0|
|[29.0,10203.18,1....|    0|[3.76523264268222...|[0.97736212052779...|       0.0|
|[29.0,11274.46,1....|    0|[4.35476865476769...|[0.98731750094735...|       0.0|
|[29.0,12711.15,0....|    0|[5.26084427574565...|[0.99483588620311...|       0.0|
|[30.0,10183.98,1....|    0|[2.84652448681721...|[0.94513875078948...|       0.0|
|[31.0,5304.6,0.

In [15]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

0.7390589863345443

In [16]:
final_lr_model = lr_churn.fit(final_data)
new_customers = spark.read.csv('gs://new_bucket_vaishu/new_customers.csv',inferSchema=True,header=True)
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [17]:
test_new_customers = assembler.transform(new_customers)
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
final_results = final_lr_model.transform(test_new_customers)
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

